# <mark>\## para saber mais</mark>

<mark>[https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlserver?view=azps-8.0.0](https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlserver?view=azps-8.0.0)  
</mark>

[https://docs.microsoft.com/en-us/azure/azure-sql/database/scripts/create-and-configure-database-powershell?view=azuresql](https://docs.microsoft.com/en-us/azure/azure-sql/database/scripts/create-and-configure-database-powershell?view=azuresql)

https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqldatabase?view=azps-8.0.0

# <mark>1) Criação de variaveis</mark>

In [19]:
$resourceGroupName="DP300"
$location ="Brazil South"
$serverName ="sql-server-dp300"
$startIp = "0.0.0.0"
$endIp = "0.0.0.0"

# <mark>2) Senhas do banco de dados integrados pelo Key Vault</mark>

In [20]:
$secretText = Get-AzKeyVaultSecret -VaultName 'key-vault-senha' -Name 'password-serverdp300' -AsPlainText
$adminSqlLogin = Get-AzKeyVaultSecret -VaultName 'key-vault-senha' -Name 'user-serverdp300' -AsPlainText


# <mark>3) Create New SQL  Server</mark>

In [22]:


$server = New-AzSqlServer -ResourceGroupName $resourceGroupName  -ServerName $serverName `
    -Location $location `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $adminSqlLogin, $(ConvertTo-SecureString -String $secretText -AsPlainText -Force))


New-AzSqlServer : Server with name: 'sql-server-dp300' already exists.
No linha:2 caractere:11
+ $server = New-AzSqlServer -ResourceGroupName $resourceGroupName  -Ser ...
+           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : CloseError: (:) [New-AzSqlServer], PSArgumentException
    + FullyQualifiedErrorId : Microsoft.Azure.Commands.Sql.Server.Cmdlet.NewAzureSqlServer
 


# <mark>4) Permitindo todos os Ips</mark>

In [23]:
# Create a server firewall rule that allows access from the specified IP range
$serverFirewallRule = New-AzSqlServerFirewallRule -ResourceGroupName $resourceGroupName `
    -ServerName $serverName `
    -FirewallRuleName "AllowedIPs" -StartIpAddress $startIp -EndIpAddress $endIp

# <mark>5) Criando um banco de dados como exemplo</mark>

In [31]:
# Create a blank database with an S0 performance level Model DTU
$database = New-AzSqlDatabase  -ResourceGroupName $resourceGroupName -ServerName $serverName -DatabaseName 'AdventureWorksLT' `
    -RequestedServiceObjectiveName "S0" `
    -SampleName "AdventureWorksLT" `
    -BackupStorageRedundancy "Local"


AVISO: Upcoming breaking changes in the cmdlet 'New-AzSqlDatabase' :
- The output type 'Microsoft.Azure.Commands.Sql.Database.Model.AzureSqlDatabaseModel' is changing
- The following properties in the output type are being deprecated : 'BackupStorageRedundancy'
- The following properties are being added to the output type : 'CurrentBackupStorageRedundancy' 
'RequestedBackupStorageRedundancy'
- The change is expected to take effect from the version : '3.0.0'
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other 
information on breaking changes in Azure PowerShell.


# <mark>\## outros exemplos de criação de banco de dados</mark>

In [ ]:
New-AzSqlDatabase -ResourceGroupName "ResourceGroup01" -ServerName "Server01" -DatabaseName "Database03" -Edition "GeneralPurpose" -Vcore 2 -ComputeGeneration "Gen5"


## <mark>Create an Serverless database on the specified server</mark>

In [ ]:
New-AzSqlDatabase -ResourceGroupName "ResourceGroup01" -ServerName "Server01" -DatabaseName "Database04" -Edition "GeneralPurpose" -Vcore 2 -ComputeGeneration "Gen5" -ComputeModel Serverless



# <mark>\## Para limpar os recursos</mark>

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourcegroupname